In [38]:
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

sns.set(style="ticks")
sns.set_style('whitegrid')
sns.set_color_codes()

import matplotlib
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import matplotlib.pyplot as plt

In [39]:
import sys
sys.path.append('C:\\Users\\dlfdus\\PycharmProjects\\mypacakge\\')
from corpus import DoublespaceLineCorpus
from corpus import DoublespaceLineCorpusWithList
from tqdm import tqdm_notebook
import requests
import pandas as pd
import re
import pickle
from collections import Counter

In [40]:
import pycrfsuite
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import PyCRFSuiteSpacing
from soynlp.tokenizer import LTokenizer
import numpy as np

In [41]:
with open('data/tmp/tokenized_reviews.pkl', 'rb') as f:
    tokenized_reviews = pickle.load(f)
with open('data/tmp/word_dictionary.pkl', 'rb') as f:
    word_dictionary = pickle.load(f)
with open('data/tmp/jobplanet_cohesionscore.pkl', 'rb') as f:
    cohesion_scores = pickle.load(f)
ltokenizer = LTokenizer(scores=cohesion_scores)

In [42]:
to_feature = CharacterFeatureTransformer(TemplateGenerator(begin=-2, 
                                                           end=2,
                                                           min_range_length=3,
                                                           max_range_length=3))
model_path = 'data/tmp/package_test.crfsuite'
correct = PyCRFSuiteSpacing(to_feature)
correct.load_tagger(model_path)

In [43]:
def tokenizer(comment):
    comment = correct(comment)
    tokenized = ''
    for word in ltokenizer.tokenize(comment):
        if word in word_dictionary and len(word)>1:
            tokenized += word
            tokenized += ' '
    tokenized = tokenized.strip()
    return tokenized

In [44]:
tokenizer('복지는좋지만분위기때문에너무힘듬')

'복지 좋지만 분위기 때문에 너무 힘듬'

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
#tfidfvectorizer 보다 countvectorizer 성능이 더 좋음 (1프로 정도)
vectorizer = CountVectorizer(lowercase=False)
X_data = vectorizer.fit_transform(tokenized_reviews)

In [47]:
with open('data/tmp/vectorizer.pkl' , 'wb') as f:
    pickle.dump(vectorizer, f)

In [48]:
Y_data = []
for i in range(len(tokenized_reviews)):
    if i < len(tokenized_reviews)/2:
        Y_data.append(0)
    else:
        Y_data.append(1)

In [13]:
X_data

<130770x15457 sparse matrix of type '<class 'numpy.int64'>'
	with 2175734 stored elements in Compressed Sparse Row format>

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.25)

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

logistic_l2 = LogisticRegression(C=1, penalty='l2')
logistic_l2.fit(X_train, y_train)

NB = BernoulliNB()
NB.fit(X_train, y_train)

MLP = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 5), random_state=1)
MLP.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [100]:
with open('data/tmp/lr.pkl' , 'wb') as f:
    pickle.dump(logistic_l2, f)

In [104]:
#with open('data/tmp/MLP.pkl' , 'wb') as f:
    #pickle.dump(MLP, f)

In [28]:
#with open('data/tmp/MLP.pkl' , 'rb') as f:
    #MLP = pickle.load(f)

In [50]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

sns.set(style="ticks")
sns.set_style('whitegrid')
sns.set_color_codes()

print("Training accuracy: {:.5f}".format(logistic_l2.score(X_train, y_train)))
print("Test accuracy: {:.5f}".format(logistic_l2.score(X_test, y_test)))
predict = logistic_l2.predict(X_test)
print(classification_report(predict, y_test), '\n')
cm = pd.DataFrame(confusion_matrix(predict, y_test), columns=['Positive','Negative'], index=['Positive','Negative'])
cm

Training accuracy: 0.95898
Test accuracy: 0.93161
             precision    recall  f1-score   support

          0       0.93      0.93      0.93     16217
          1       0.93      0.93      0.93     16476

avg / total       0.93      0.93      0.93     32693
 



,Positive,Negative
Positive,15142,1075
Negative,1161,15315


In [51]:
sns.set(style="ticks")
sns.set_style('whitegrid')
sns.set_color_codes()

print("Training accuracy: {:.5f}".format(NB.score(X_train, y_train)))
print("Test accuracy: {:.5f}".format(NB.score(X_test, y_test)))
predict = NB.predict(X_test)
print(classification_report(predict, y_test), '\n')
cm = pd.DataFrame(confusion_matrix(predict, y_test), columns=['Positive','Negative'], index=['Positive','Negative'])
cm

Training accuracy: 0.91510
Test accuracy: 0.90799
             precision    recall  f1-score   support

          0       0.93      0.89      0.91     17005
          1       0.89      0.93      0.91     15688

avg / total       0.91      0.91      0.91     32693
 



,Positive,Negative
Positive,15150,1855
Negative,1153,14535


In [36]:
sns.set(style="ticks")
sns.set_style('whitegrid')
sns.set_color_codes()

print("Training accuracy: {:.5f}".format(MLP.score(X_train, y_train)))
print("Test accuracy: {:.5f}".format(MLP.score(X_test, y_test)))
predict = MLP.predict(X_test)
print(classification_report(predict, y_test), '\n')
cm = pd.DataFrame(confusion_matrix(predict, y_test), columns=['Positive','Negative'], index=['Positive','Negative'])
cm

Training accuracy: 0.97670
Test accuracy: 0.97709
             precision    recall  f1-score   support

          0       0.98      0.98      0.98     16268
          1       0.98      0.98      0.98     16425

avg / total       0.98      0.98      0.98     32693
 



,Positive,Negative
Positive,15911,357
Negative,392,16033


In [57]:
def sentiment_analysis(model = logistic_l2, comment=0, direct_input=True, print_tokenized_text=False):
    if direct_input == True:
        comment = input()
    tokenized_comment = [tokenizer(comment)]
    if print_tokenized_text == True:
        print(' -> ',tokenizer(comment))
    vectorized_coemment = vectorizer.transform(tokenized_comment).toarray()
    predict = model.predict(vectorized_coemment)
    predict_proba = model.predict_proba(vectorized_coemment)
    print(' ')
    if predict[0] == 0:
        print('긍정 문장\n')
    else:
        print('부정 문장\n')
        
    print('긍정 : {:.4f}\n부정 : {:.4f}'.format(predict_proba[0][0], predict_proba[0][1]))

In [106]:
sentiment_analysis()

정시퇴근 절대 못함
 
부정 문장

긍정 : 0.0000
부정 : 1.0000


In [121]:
sentiment_analysis(model = logistic_l2)

정시퇴근 절대 못함
 
부정 문장

긍정 : 0.1424
부정 : 0.8576


In [25]:
sentiment_analysis()

업무보다는 정치질이 우선인 회사 발전 가능성이 없음
 
부정 문장

긍정 : 0.1399
부정 : 0.8601


In [26]:
sentiment_analysis()

복지와 연봉 모두 최고 자부심이 느껴짐
 
긍정 문장

긍정 : 0.9361
부정 : 0.0639


In [27]:
sentiment_analysis()

대기업이지만 복지와 연봉이 좋지 않음
 
부정 문장

긍정 : 0.1120
부정 : 0.8880


In [28]:
sentiment_analysis()

성장 가능성이 낮음
 
부정 문장

긍정 : 0.3959
부정 : 0.6041


In [30]:
sentiment_analysis()

연봉 좋고 복지는 좋지만 절대 다니고 싶지 않은 회사
 
긍정 문장

긍정 : 0.7578
부정 : 0.2422


In [31]:
sentiment_analysis()

연봉은 좋고 복지는 좋지만 꼭 다니고 싶은 회사
 
긍정 문장

긍정 : 0.9234
부정 : 0.0766


In [38]:
sentiment_analysis()

연봉 좋고 복지는 좋지만 쓰레기같은 회사 절대 다니면 안됨 최악
 
부정 문장

긍정 : 0.0585
부정 : 0.9415


In [42]:
sentiment_analysis()

연봉 좋음 연봉 최악
 
긍정 문장

긍정 : 0.5960
부정 : 0.4040


In [43]:
sentiment_analysis()

최고 최악
 
부정 문장

긍정 : 0.3793
부정 : 0.6207


In [45]:
sentiment_analysis()

최악 최고
 
부정 문장

긍정 : 0.3793
부정 : 0.6207


In [47]:
sentiment_analysis()

최고 최악 최고
 
긍정 문장

긍정 : 0.6945
부정 : 0.3055


In [85]:
sentiment_analysis()

연봉이 높습니다. 정시퇴근을 합니다.
 
긍정 문장

긍정 : 0.7406
부정 : 0.2594


In [86]:
sentiment_analysis()

연봉이 낮기도 하고 연봉이 높기도 합니다. 정시퇴근을 안할 때도 있고 정시퇴근을 할 때도 있습니다.
 
부정 문장

긍정 : 0.2293
부정 : 0.7707


In [87]:
sentiment_analysis()

연봉이 높기도 하고 연봉이 낮기도 합니다. 정시퇴근을 할 때도 있고 정시퇴근을 안할 때도 있습니다.
 
부정 문장

긍정 : 0.2293
부정 : 0.7707


In [50]:
sentences = ['주 5일제 연장근무 수당 30분 단위 지급.칼퇴 가능. 칼출근도 가능.진급 가능. 진급시 신세계 정직원. 내부채용으로 사무직도 지원가능하나 생각보다 매장직 근무자들이 해외 출신이 많으며 고스펙자들이 많음.',
            '같은 임금이지만 매장에 따라 노동강도가 차이가 많이난다.',
            '합리적인 복지 보상체계 부족 ..그닥 딱히 쓸만한 것이 없는 것같다',
            '체계적인 시스템에서 업무 습득이 가능, 수평적인 분위기',
            '수평적 관계로 사람들 모두가 좋음 칼퇴근가능 새로운 것에 대한 도전에 두려움 없음',
            '직원이 많지 않아 스스로 해결해야 하는 일이 많다. 직원들은 수평적이지만 대표님은 권위적일 수 있음']

In [70]:
for sentence in sentences:
    print(sentence, '\n')
    sentiment_analysis(sentence, direct_input=False, print_tokenized_text=True)
    print('----------------------------------------------------')

주 5일제 연장근무 수당 30분 단위 지급.칼퇴 가능. 칼출근도 가능.진급 가능. 진급시 신세계 정직원. 내부채용으로 사무직도 지원가능하나 생각보다 매장직 근무자들이 해외 출신이 많으며 고스펙자들이 많음. 

 ->  5일 연장근무 수당 30분 단위 지급 가능 칼출근 가능 가능 진급 신세계 정직원 내부 채용 으로 사무직 지원 가능 하나 생각 보다 매장 근무 자들이 해외 출신이 많으며 들이 많음
 
긍정 문장

긍정 : 0.9967
부정 : 0.0033
----------------------------------------------------
같은 임금이지만 매장에 따라 노동강도가 차이가 많이난다. 

 ->  같은 임금 이지만 매장에 따라 노동강도가 차이가 많이 난다
 
부정 문장

긍정 : 0.0430
부정 : 0.9570
----------------------------------------------------
합리적인 복지 보상체계 부족 ..그닥 딱히 쓸만한 것이 없는 것같다 

 ->  합리적인 복지 보상체계 부족 그닥 딱히 쓸만한 것이 없는 같다
 
부정 문장

긍정 : 0.0562
부정 : 0.9438
----------------------------------------------------
체계적인 시스템에서 업무 습득이 가능, 수평적인 분위기 

 ->  체계 적인 시스템 에서 업무 습득 가능 수평적인 분위기
 
긍정 문장

긍정 : 0.9926
부정 : 0.0074
----------------------------------------------------
수평적 관계로 사람들 모두가 좋음 칼퇴근가능 새로운 것에 대한 도전에 두려움 없음 

 ->  수평적 관계 사람들 모두 좋음 칼퇴 가능 새로운 것에 대한 도전 없음
 
긍정 문장

긍정 : 0.9984
부정 : 0.0016
----------------------------------------------------
직원이 많지 않아 스스로 해결해야 하는 일이 많다. 직원들은 수

In [86]:
sentiment_analysis(print_tokenized_text=False)

일이 많음^^
 
부정 문장

긍정 : 0.3448
부정 : 0.6552


In [58]:
sentiment_analysis()

다좋은데 애들이 더럽다
 
부정 문장

긍정 : 0.3586
부정 : 0.6414


In [59]:
sentiment_analysis()

다좋은데 애들이 너무 더럽다
 
부정 문장

긍정 : 0.0864
부정 : 0.9136


In [61]:
sentiment_analysis()

연봉이 높다
 
긍정 문장

긍정 : 0.6220
부정 : 0.3780


In [65]:
sentiment_analysis()

연봉이 너무 높다
 
부정 문장

긍정 : 0.2178
부정 : 0.7822


In [68]:
sentiment_analysis()

연봉이 진짜 높아서 너무 좋다
 
긍정 문장

긍정 : 0.7870
부정 : 0.2130


In [70]:
sentiment_analysis()

연봉이 진짜 높아서 좋다
 
긍정 문장

긍정 : 0.9562
부정 : 0.0438
